In [ ]:
import os
import time
from typing import Dict, Any

# === CrewAI imports ===
# Agent: Represents an autonomous worker with a role, goal, and style
# Task:  Represents a single instruction the agent must complete
# Crew:  Orchestrates one or more agents + tasks
# Process: Defines how tasks run (sequential / parallel)
from crewai import Agent, Task, Crew, Process

# === LLM backend ===
# ChatOllama lets LangChain talk to a local Ollama server
# LLM is CrewAI's wrapper around LangChain LLM objects
from langchain_community.chat_models import ChatOllama
from crewai import LLM


# ---------------------------------------------------------------------
# 🧠 1. LOAD THE LANGUAGE MODEL
# ---------------------------------------------------------------------
# We wrap a real LangChain model (`ollama/llama3`) inside CrewAI's LLM class.
# This ensures CrewAI can talk to local Ollama without custom plumbing.
llm = LLM(model="ollama/llama3")  



# ---------------------------------------------------------------------
# 🧩 2. SYSTEM PERSONA — A HIGH-LEVEL "CONTEXT SETTER"
# ---------------------------------------------------------------------
# This tells the agent how it should behave *across all tasks*.
# It is not the task instruction — it defines personality, tone, and guardrails.
SYSTEM = (
    "You are Captain Byte, a friendly, vivid cyber-adventure guide. "
    "Your mission is to turn technical ideas into short, exciting stories "
    "WITHOUT breaking scientific accuracy. "
    "Avoid hallucinations, avoid adding facts you are unsure of, "
    "and keep explanations simple for beginners."
)



# ---------------------------------------------------------------------
# 🧑‍🚀 3. DEFINE THE AGENT
# ---------------------------------------------------------------------
# An agent is like a self-contained worker with:
#   - role:     What type of expert it is
#   - goal:     What ultimate objective it always tries to achieve
#   - backstory:Fun flavor text (not required, but improves style)
#
# allow_delegation=False means the agent cannot spawn new agents on its own.
# verbose=True prints logs so you can debug each step.
agent = Agent(
    role="Cyber Adventure Guide",
    goal="Explain complex topics as short adventure stories while staying factual.",
    backstory="Captain Byte sails the Digital Seas, helping newcomers understand technology.",
    llm=llm,
    allow_delegation=False,
    verbose=True,
)



# ---------------------------------------------------------------------
# 🔎 4. A TINY FACT LIBRARY (NO MEMORY, PURELY STATELESS)
# ---------------------------------------------------------------------
# This simulates a “tool” the agent could call. 
# In real Agentic AI, tools can include: databases, scrapers, calculators, APIs, etc.
#
# Here we keep it simple: the tool returns a tiny verified fact.
def lore_lookup(topic: str) -> str:
    data = {
        "encryption": "Encryption transforms readable text into scrambled ciphertext using mathematical keys.",
        "black holes": "Black holes contain an event horizon where gravity is strong enough that not even light escapes.",
    }
    return data.get(topic.lower(), f"No verified nugget was found for '{topic}'.")



# ---------------------------------------------------------------------
# 🚀 5. EXECUTE ONE COMPLETE RUN
# ---------------------------------------------------------------------
# This function:
#   - Gathers context
#   - Builds a robust prompt
#   - Creates a Crew task
#   - Executes it
#   - Returns structured output
def run_once(topic: str) -> Dict[str, Any]:

    # Step 1 — Get a tiny verified fact
    nugget = lore_lookup(topic)

    # Step 2 — Some pre-selected analogies beginner explanations often use
    analogy_library = ["lock & key", "secret courier", "signal beacon"]

    # Step 3 — Add transparent context to the prompt
    # (Transparency is important in agentic systems so reasoning is reproducible.)
    context = (
        f"# CONTEXT\n"
        f"- Topic: {topic}\n"
        f"- Verified Fact (Nugget): {nugget}\n"
        f"- Analogy Options: {analogy_library}\n"
    )

    # ------------------------------------------------------------------
    # 🎯 6. ROBUST USER PROMPT
    # ------------------------------------------------------------------
    # This prompt is structured, constrained, and defensive against hallucinations.
    # It includes:
    #   - persona grounding
    #   - context grounding
    #   - strict length control
    #   - accuracy reminders
    #   - step-by-step clarity
    user_prompt = f"""{SYSTEM}
                    
                    {context}
                    
                    # TASK
                    Explain the topic: **"{topic}"** as a short adventure story for a curious beginner.
                    
                    ## Requirements
                    - Use **two short paragraphs**.
                    - Maximum **150 words total**.
                    - Must contain a **vivid adventure theme** BUT remain **scientifically accurate**.
                    - Only use facts you are **confident** about.  
                    - If the nugget contains limited information, keep the story simple instead of inventing details.
                    - Prefer using one analogy from the provided list if relevant.
                    
                    Begin now.
                    """

    # ------------------------------------------------------------------
    # 📝 7. CREATE A CREW TASK
    # ------------------------------------------------------------------
    task = Task(
        description=user_prompt,
        expected_output="Two short paragraphs (≤150 words), vivid but factually correct.",
        agent=agent,
    )

    # ------------------------------------------------------------------
    # 🧠 8. RUN THE CREW
    # ------------------------------------------------------------------
    crew = Crew(
        agents=[agent],
        tasks=[task],
        process=Process.sequential,  # Only one task so sequential is simplest
        verbose=True
    )

    # Execute the agent's reasoning + generation
    output = crew.kickoff()

    return {"topic": topic, "text": str(output)}



# ---------------------------------------------------------------------
# 🏁 9. MAIN ENTRYPOINT
# ---------------------------------------------------------------------
# This section runs only if you call `python script.py`
if __name__ == "__main__":
    t0 = time.time()

    # Read topic from environment variable — useful for Docker/Kubernetes
    topic = os.getenv("TOPIC", "encryption")

    print(f"\n=== Captain Byte (Stateless / Ollama) on '{topic}' ===\n")

    result = run_once(topic)

    print("\n--- Output ---\n")
    print(result["text"])

    # Simple timing stats
    dt = time.time() - t0
    print("\n--- Stats ---")
    print({"elapsed_s": round(dt, 2)})
